In [1]:
import sys
import os
import numpy as np
import numba
import h5py
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline 
%config InlineBackend.figure_format = 'svg'
%config InlineBackend.figure_format = 'png'



from colossus.lss import bias
from colossus.lss import mass_function
from colossus.cosmology import cosmology
cosmology.setCosmology('planck15');

from astropy.modeling import models, fitting
import Pk_library as PKL


import sys, os
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')
# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

# bPe/Pe_mean 

## mag

In [ ]:
box_names   = ["Box0", "Box0_meshes",  "Box1a",  "Box2b"]
meshfile_names  = ["pe_meshes_0", "mag_pe_den_meshes_0","pe_meshes_0", "pe_meshes_0"]
snap_no_mags= [np.arange(3, 15), np.array([6,7,8,9,10,11,12,14,17,21,25,29, 33, 37]), np.arange(2, 8), np.arange(7, 21)]
def mev_cm3_2_Pa():
    kg2ev_c2 = 1/(1.78*10**-36) #ev/c^2
    c_speed = 3*10**5
    h = 0.67
    return 10**29 *h**2/(kg2ev_c2/c_speed**2 *h**2 *10**3) /10**17

redshifts_mag = []
Pe_means_mag = []
bPes_mag = []
for i in range(4):
    box_name = box_names[i]
    meshfile_name = meshfile_names[i]
    snap_no_mag = snap_no_mags[i]
    redshift = np.zeros(len(snap_no_mag))
    Pe_mean = np.zeros(len(snap_no_mag))
    bPe = np.zeros(len(snap_no_mag))
    
    for j in range(len(snap_no_mag)):
        
        file_name = "/u/ziyang/data/Magneticum/"+box_name+"/"+meshfile_name+str(snap_no_mag[j]).zfill(2)+".hdf5"
        f = h5py.File(file_name)
        #print(dict(f), f.attrs.keys())

        if box_name == "Box0_meshes":
            Boxsize = f['Header'].attrs['BoxSize']
            redshift[j] = f['Header'].attrs["Redshift"]
        else:
            Boxsize = f.attrs["BoxLength"]
            redshift[j] = f.attrs["Redshift"]
        #print(snap_no_mag[j], redshift[j])
        #continue
        Pe = f["ElectronPressure"][:]
        M = f["MatterDensity"][:]
        #Pe = Pe/np.mean(Pe)
        M = M/np.mean(M)
        Pe_mean[j] = np.mean(Pe)/mev_cm3_2_Pa()
        
        blockPrint()
        Pk = PKL.XPk([M, Pe], Boxsize, axis=0, MAS=['CIC','CIC'], threads=1)
        k_mag = Pk.k3D
        pk_mm_mag=Pk.Pk[:,0,0]
        pk_pepe_mag=Pk.Pk[:,0,1]
        pk_pem_mag=Pk.XPk[:,0,0]
        enablePrint()

        label = np.where(k_mag<0.03)
        bPe[j] = np.mean((pk_pem_mag/pk_mm_mag)[label])/mev_cm3_2_Pa()
    
    redshifts_mag.append(redshift)
    Pe_means_mag.append(Pe_mean)
    bPes_mag.append(bPe)
    
np.savez("bPe_Pe_mean_mag.npz", simu_box = box_names, snap_no_mag=snap_no_mags, redshifts=redshifts_mag, Pe_means=Pe_means_mag, bPes=bPes_mag)

In [11]:
d = np.load("bPe_Pe_mean_mag.npz", allow_pickle=True)
d["simu_box"]

array(['Box0', 'Box0_meshes', 'Box1a', 'Box2b'], dtype='<U11')

## MTNG

In [2]:
#MTNG
k_cut = 0.18
degree = 2
MAS = "NGP"
grid = 128
datadir = "/u/ziyang/data/TSZ_MTNG/PK_deltape/"
simu_names = ["MTNG-L125-1080-A", "MTNG-L31.25-270-A", "MTNG-L500-4320-A", "MTNG-L62.5-540-A"]

redshift2snapshot = {4:80, 3:94, 2:129, 1:179, 0.5:214, 0:264}
redshifts = np.array([0, 0.5, 1, 2, 3, 4])
simu_name = simu_names[2]


by = np.zeros(len(redshifts))
Pe_simu = np.zeros(len(redshifts))

for i in [0,1,2,3,4, 5]:
    snap_no = redshift2snapshot[redshifts[i]]
    dataname = datadir+"PK_deltape_"+simu_name+"_"+str(snap_no)+"_"+MAS+"_"+str(grid)+".npz"
    d=np.load(dataname)
    k = d["k"]
    ratio = d["Pk_mPe"]/d["Pk_mm"]
    Pe_simu[i] = d["Pe_mean"]


    model_poly = models.Polynomial1D(degree=degree)
    fitter_poly = fitting.LinearLSQFitter()
    label = np.where(k<k_cut)[0]
    best_fit_poly = fitter_poly(model_poly, k[label]**2, ratio[label])#, weights = (N_mode[label]))
    c0 = best_fit_poly.c0.value
    by[i] = c0
np.savez("bPe_Pe_mean_MTNG.npz",  redshift2snapshot=redshift2snapshot,
         redshifts=redshifts, Pe_means=Pe_simu, by=by)

## ITNG

In [12]:
#ITNG
k_cut = 0.4
degree = 2
datadir = "/u/ziyang/data/TSZ_MTNG/PK_deltape/"
simu_names = ["L205n1250TNG","L205n2500TNG","L205n625TNG", "L75n1820TNG", "L75n455TNG", "L75n910TNG"]
k_cuts = [0.18, 0.18, 0.18, 0.5, 0.5, 0.5]
redshift2snapshot = {3:25, 2:33, 1:50, 0.5:67, 0:99}
redshifts = np.array([0, 0.5, 1, 2, 3])
MAS = "NGP"
grid=128
#simu_name = simu_names[1]

Pe_simu = np.zeros((len(simu_names),len(redshifts)))
by = np.zeros((len(simu_names),len(redshifts)))
for j in range(len(simu_names)):
    simu_name = simu_names[j]
    k_cut = k_cuts[j]

    for i in [0,1,2,3,4]:
        snap_no = redshift2snapshot[redshifts[i]]

        dataname = datadir+"PK_deltape_"+simu_name+"_"+str(snap_no)+"_"+MAS+"_"+str(grid)+".npz"
        d=np.load(dataname)
        k=d["k"]
        ratio = d["Pk_mPe"]/d["Pk_mm"]
        Pe_simu[j,i] = d["Pe_mean"]

        model_poly = models.Polynomial1D(degree=degree)
        fitter_poly = fitting.LinearLSQFitter()
        label = np.where(k<k_cut)[0]
        best_fit_poly = fitter_poly(model_poly, k[label]**2, ratio[label])#, weights = (N_mode[label]))
        c0 = best_fit_poly.c0.value
        by[j, i] = c0

np.savez("bPe_Pe_mean_ITNG.npz",  redshift2snapshot=redshift2snapshot,simu_names=simu_names,
         redshifts=redshifts, Pe_means=Pe_simu, by=by)